# Import Libraries

In [56]:
import pandas as pd
import os
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import requests
import csv
from io import StringIO
from sklearn.utils import resample
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Data Loading

In [57]:
folder_path = "dataset"

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

# List to store each DataFrame
dfs = []

# Read and collect DataFrames
for file in csv_files:
    full_path = os.path.join(folder_path, file)
    print(f"Reading: {full_path}")
    df = pd.read_csv(full_path)
    # print(df.head(5))
    dfs.append(df)

# Combine all into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Show result
print(combined_df.head())
print(f"\nTotal combined rows: {len(combined_df)}")

Reading: dataset/Rok_Cargo_Wanita_Span_Panjang_Tebal_Skirt_Bawahan__reviews.csv
Reading: dataset/Eiji_-_Flare_Pants_-_Highwaist_Flare_Pants_-_Celan_reviews.csv
Reading: dataset/CELANA_HIGHWAIST_LOOSE_JEANS_WANITA___BAGGY_JEANS__reviews.csv
Reading: dataset/SEPTEMBER_TOP___Atasan_Knit_Wanita_Korean_Top_Baju_reviews.csv
Reading: dataset/Apple iPhone 14 - Garansi Resmi Apple Indonesia - _reviews.csv
Reading: dataset/Dress_Kaftan_Batik_Elegan_Wanita_Pesta_Aruna_-_Nav_reviews.csv
Reading: dataset/Celana_Panjang_Jogger_Training_Olahraga_Lari_Gym_J_reviews.csv
Reading: dataset/Nuevo_Velocity_Putih_Silver_Sepatu_Sneakers_Runnin_reviews.csv
Reading: dataset/ROK_RAJUT_SPAN_7_8_SE_BETIS_ROK_KERJA_FORMAL_KEKIN_reviews.csv
Reading: dataset/MARCH_TOP___Atasan_Knit_Wanita_Korean_Top_Baju_Kni_reviews.csv
Reading: dataset/Delibra_Jaket_Bomber_Suede_Pria_Premium_-_Black,_X_reviews.csv
Reading: dataset/DIANS_-_Rok_Span_Panjang_Scuba_Premium_Rok_Pensil__reviews.csv
Reading: dataset/Celana_Jeans_Pria_celan

In [58]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12170 entries, 0 to 12169
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  10718 non-null  object
 1   rating  12170 non-null  object
 2   date    10870 non-null  object
dtypes: object(3)
memory usage: 285.4+ KB


# Data Preprocessing

### Helper Functions

In [59]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka
 
    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks

    # remove emojis
    emoji_pattern = re.compile(
        "["                               
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002700-\U000027BF"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    
    text = emoji_pattern.sub(r'', text)
    
    return text
 
def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text
 
# def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
#     text = word_tokenize(text)
#     return text
 
def tokenizingText(text):
    if pd.isna(text):
        return []
    return text.split()

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    # listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords)
    # listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','nya','na','sih','ku',"di","ya","loh","kah","woi","woii","woy", "nih", "trus", "tuh",\
                          "yah", "ajah", "lagi", "lah", "aj", "aja", "jg", "juga", "jga", "yng", 'apa', "cuman", "deh",\
                            "min", "gak"\
                            "si", "an"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text
 
def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
 
    # Memecah teks menjadi daftar kata
    words = text.split()
 
    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]
 
    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)
 
    return stemmed_text
 
def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [ ]:
# Download slang dictionary from a public dataset (if available)
url = "https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/refs/heads/master/combined_slang_words.txt"
slang_dict = requests.get(url).json()

slang_dict['aku'] = 'saya'
slang_dict['ak'] = 'saya'
slang_dict['gua'] = 'saya'
slang_dict['gw'] = 'saya'

slang_dict['jlk'] = 'jelek'
slang_dict['jlek'] = 'jelek'
slang_dict['burik'] = 'jelek'
slang_dict['buriq'] = 'jelek'
slang_dict['ampas'] = 'jelek'
slang_dict['amps'] = 'jelek'
slang_dict['buruk'] = 'jelek'
slang_dict['kentang'] = 'jelek'
slang_dict['bobrok'] = 'jelek'

slang_dict['bgs'] = 'bagus'
slang_dict['bgus'] = 'bagus'
slang_dict['baguss'] = 'bagus'

slang_dict['trnyata'] = 'ternyata'

slang_dict['bgt'] = 'banget'

slang_dict['kren'] = 'keren'

slang_dict['udh'] = 'udah'

slang_dict['kasi'] = 'kasih'
slang_dict['ksi'] = 'kasih'
slang_dict['ksih'] = 'kasih'

slang_dict['gk'] = 'gak'
slang_dict['ga'] = 'gak'
slang_dict['gaa'] = 'gak'
slang_dict['kagak'] = 'gak'
slang_dict['kgk'] = 'gak'
slang_dict['g'] = 'gak'
slang_dict['engga'] = 'gak'
slang_dict['tdk'] = 'gak'
slang_dict['nggk'] = 'gak'

slang_dict['jls'] = 'jelas'
slang_dict['jlas'] = 'jelas'
slang_dict['danta'] = 'jelas'

slang_dict['mntp'] = 'mantap'
slang_dict['mantul'] = 'mantap'
slang_dict['mntap'] = 'mantap'

slang_dict['lg'] = 'lagi'
slang_dict['lgi'] = 'lagi'

slang_dict['uk'] = 'ukuran'

slang_dict['ksel'] = 'kesal'
slang_dict['kesel'] = 'kesal'
slang_dict['sebel'] = 'kesal'
slang_dict['sebal'] = 'kesal'

slang_dict['bacod'] = 'bacot'
slang_dict['bct'] = 'bacot'
slang_dict['bcd'] = 'bacot'

slang_dict['goblog'] = 'goblok'
slang_dict['gblg'] = 'goblok'
slang_dict['gblk'] = 'goblok'
slang_dict['bego'] = 'goblok'
slang_dict['bgo'] = 'goblok'
slang_dict['tolol'] = 'goblok'
slang_dict['tlol'] = 'goblok'
slang_dict['idiot'] = 'goblok'

slang_dict['trun'] = 'turun'

slang_dict['yt'] = 'youtube'

slang_dict['ajg'] = 'anjing'
slang_dict['anj'] = 'anjing'
slang_dict['anjg'] = 'anjing'
slang_dict['anjir'] = 'anjing'
slang_dict['anjr'] = 'anjing'

slang_dict['lemod'] = 'lemot'

slang_dict['pass'] = 'pas'

slang_dict['gaje'] = 'aneh'

slang_dict['wr'] = 'win rate'
slang_dict['winrate'] = 'win rate'
slang_dict['ws'] = 'win streak'
slang_dict['winstreak'] = 'win streak'

slang_dict['leg'] = 'lag'
slang_dict['ngeleg'] = 'ngelag'

slang_dict['hps'] = 'hapus'
slang_dict['hpus'] = 'hapus'
slang_dict['uninstal'] = 'hapus'
slang_dict['uninstall'] = 'hapus'

# Bi-gram
slang_dict['gak jelas'] = 'aneh'
slang_dict['ga jelas'] = 'aneh'
slang_dict['ngga jelas'] = 'aneh'
slang_dict['g jelas'] = 'aneh'

def fix_slangwords(text):
    words = text.lower().split()
    
    # Step 1: Fix unigrams
    fixed_unigrams = [slang_dict.get(word, word) for word in words]
    
    # Step 2: Check for fixed bigrams
    i = 0
    final_words = []
    while i < len(fixed_unigrams):
        if i + 1 < len(fixed_unigrams):
            bigram = f"{fixed_unigrams[i]} {fixed_unigrams[i+1]}"
            if bigram in slang_dict:
                final_words.append(slang_dict[bigram])
                i += 2
                continue
        
        final_words.append(fixed_unigrams[i])
        i += 1

    return ' '.join(final_words)

In [61]:
# Remove missing values before pre-processing
combined_df.dropna(subset=['review'], inplace=True)

In [62]:
def preprocess_review(review_df):
    # Membersihkan teks dan menyimpannya di kolom 'text_clean'
    review_df['text_clean'] = review_df['review'].apply(cleaningText)
    
    # Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
    review_df['text_casefoldingText'] = review_df['text_clean'].apply(casefoldingText)
    
    # Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
    review_df['text_slangwords'] = review_df['text_casefoldingText'].apply(fix_slangwords)

    # Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
    review_df['text_tokenizingText'] = review_df['text_slangwords'].apply(tokenizingText)
    
    # Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
    review_df['text_stopword'] = review_df['text_tokenizingText'].apply(filteringText)
    
    # Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
    review_df['text_akhir'] = review_df['text_stopword'].apply(toSentence)

    return review_df

In [63]:
preprocessed_df = preprocess_review(combined_df)

In [64]:
preprocessed_df.head()

,review,rating,date,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,"Bagus banget sesuai banget sama uk badan aku ,...",bintang 5,4 minggu lalu,Bagus banget sesuai banget sama uk badan aku b...,bagus banget sesuai banget sama uk badan aku b...,bagus banget sesuai banget sama uk badan saya ...,"[bagus, banget, sesuai, banget, sama, uk, bada...","[bagus, banget, sesuai, banget, uk, badan, bah...",bagus banget sesuai banget uk badan bahan paka...
1,bagus banget paketnya aku sangat suka aku kira...,bintang 5,2 bulan lalu,bagus banget paketnya aku sangat suka aku kira...,bagus banget paketnya aku sangat suka aku kira...,bagus banget paketnya saya sangat suka saya ki...,"[bagus, banget, paketnya, saya, sangat, suka, ...","[bagus, banget, paketnya, suka, rok, gak, kare...",bagus banget paketnya suka rok gak karetnya pa...
2,Untuk bahannya nyaman enak dan pas untuk tingg...,bintang 5,2 bulan lalu,Untuk bahannya nyaman enak dan pas untuk tingg...,untuk bahannya nyaman enak dan pas untuk tingg...,untuk bahannya nyaman enak dan pas untuk tingg...,"[untuk, bahannya, nyaman, enak, dan, pas, untu...","[bahannya, nyaman, enak, pas, si, panjangan, n...",bahannya nyaman enak pas si panjangan ngk papa...
3,"UK pass, cuma agak tipis bahannyaa sesuai seeh...",bintang 5,3 bulan lalu,UK pass cuma agak tipis bahannyaa sesuai seehh...,uk pass cuma agak tipis bahannyaa sesuai seehh...,uk pass cuma agak tipis bahannyaa sesuai seehh...,"[uk, pass, cuma, agak, tipis, bahannyaa, sesua...","[uk, pass, tipis, bahannyaa, sesuai, seehh, ha...",uk pass tipis bahannyaa sesuai seehh harga seg...
4,"Trnyata bahan nya TDK melar,,bahan e kasar,mak...",bintang 4,1 bulan lalu,Trnyata bahan nya TDK melarbahan e kasarmakany...,trnyata bahan nya tdk melarbahan e kasarmakany...,trnyata bahan nya tidak melarbahan e kasarmaka...,"[trnyata, bahan, nya, tidak, melarbahan, e, ka...","[trnyata, bahan, melarbahan, e, kasarmakanya, ...",trnyata bahan melarbahan e kasarmakanya hrga m...


# Labeling

In [65]:
# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()
 
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub
 
if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma
 
    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")
 
# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()
 
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub
 
if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma
 
    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [66]:
# Fungsi untuk menentukan polaritas sentimen dari tweet
 
def sentiment_analysis_lexicon_indonesia(text):
    score = 0
    # Inisialisasi skor sentimen ke 0
 
    for word in text:
        # Mengulangi setiap kata dalam teks
 
        if (word in lexicon_positive):
            score = score + lexicon_positive[word]/5.
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen
 
    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)
 
        if (word in lexicon_negative):
            score = score + lexicon_negative[word]/5.
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen
 
    polarity=''
    # Inisialisasi variabel polaritas
 
    if (score >= 0.2):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
    elif (score <= -0.2):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif
    else:
        polarity = 'neutral'
    # Ini adalah bagian yang bisa digunakan untuk menentukan polaritas netral jika diperlukan
 
    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [75]:
results = preprocessed_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
preprocessed_df['polarity_score'] = results[0]
preprocessed_df['label'] = results[1]

In [76]:
preprocessed_df.head()

,review,rating,date,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,label,polarity_score
0,"Bagus banget sesuai banget sama uk badan aku ,...",bintang 5,4 minggu lalu,Bagus banget sesuai banget sama uk badan aku b...,bagus banget sesuai banget sama uk badan aku b...,bagus banget sesuai banget sama uk badan saya ...,"[bagus, banget, sesuai, banget, sama, uk, bada...","[bagus, banget, sesuai, banget, uk, badan, bah...",bagus banget sesuai banget uk badan bahan paka...,neutral,-0.2
1,bagus banget paketnya aku sangat suka aku kira...,bintang 5,2 bulan lalu,bagus banget paketnya aku sangat suka aku kira...,bagus banget paketnya aku sangat suka aku kira...,bagus banget paketnya saya sangat suka saya ki...,"[bagus, banget, paketnya, saya, sangat, suka, ...","[bagus, banget, paketnya, suka, rok, gak, kare...",bagus banget paketnya suka rok gak karetnya pa...,positive,1.2
2,Untuk bahannya nyaman enak dan pas untuk tingg...,bintang 5,2 bulan lalu,Untuk bahannya nyaman enak dan pas untuk tingg...,untuk bahannya nyaman enak dan pas untuk tingg...,untuk bahannya nyaman enak dan pas untuk tingg...,"[untuk, bahannya, nyaman, enak, dan, pas, untu...","[bahannya, nyaman, enak, pas, si, panjangan, n...",bahannya nyaman enak pas si panjangan ngk papa...,positive,3.2
3,"UK pass, cuma agak tipis bahannyaa sesuai seeh...",bintang 5,3 bulan lalu,UK pass cuma agak tipis bahannyaa sesuai seehh...,uk pass cuma agak tipis bahannyaa sesuai seehh...,uk pass cuma agak tipis bahannyaa sesuai seehh...,"[uk, pass, cuma, agak, tipis, bahannyaa, sesua...","[uk, pass, tipis, bahannyaa, sesuai, seehh, ha...",uk pass tipis bahannyaa sesuai seehh harga seg...,positive,1.4
4,"Trnyata bahan nya TDK melar,,bahan e kasar,mak...",bintang 4,1 bulan lalu,Trnyata bahan nya TDK melarbahan e kasarmakany...,trnyata bahan nya tdk melarbahan e kasarmakany...,trnyata bahan nya tidak melarbahan e kasarmaka...,"[trnyata, bahan, nya, tidak, melarbahan, e, ka...","[trnyata, bahan, melarbahan, e, kasarmakanya, ...",trnyata bahan melarbahan e kasarmakanya hrga m...,negative,-0.4


In [77]:
preprocessed_df[preprocessed_df['label']=='negative'].head()

,review,rating,date,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,label,polarity_score
4,"Trnyata bahan nya TDK melar,,bahan e kasar,mak...",bintang 4,1 bulan lalu,Trnyata bahan nya TDK melarbahan e kasarmakany...,trnyata bahan nya tdk melarbahan e kasarmakany...,trnyata bahan nya tidak melarbahan e kasarmaka...,"[trnyata, bahan, nya, tidak, melarbahan, e, ka...","[trnyata, bahan, melarbahan, e, kasarmakanya, ...",trnyata bahan melarbahan e kasarmakanya hrga m...,negative,-0.4
10,ini gimana sih saya pesen 2 tapi malah datang ...,bintang 1,2 bulan lalu,ini gimana sih saya pesen tapi malah datang ...,ini gimana sih saya pesen tapi malah datang ...,ini bagaimana sih saya pesen tapi bahkan datan...,"[ini, bagaimana, sih, saya, pesen, tapi, bahka...","[pesen, mengecewakan]",pesen mengecewakan,negative,-0.6
11,Bahannya bagus harganya juga terjangkau\nTp ag...,bintang 5,3 bulan lalu,Bahannya bagus harganya juga terjangkau Tp aga...,bahannya bagus harganya juga terjangkau tp aga...,bahannya bagus harganya juga terjangkau tapi a...,"[bahannya, bagus, harganya, juga, terjangkau, ...","[bahannya, bagus, harganya, terjangkau, kegede...",bahannya bagus harganya terjangkau kegedean ka...,negative,-0.6
14,Bagus pas lagi 👍 the best TPI bahan nya kaku,bintang 4,3 bulan lalu,Bagus pas lagi the best TPI bahan nya kaku,bagus pas lagi the best tpi bahan nya kaku,bagus pas lagi the best tapi bahan nya kaku,"[bagus, pas, lagi, the, best, tapi, bahan, nya...","[bagus, pas, the, best, bahan, kaku]",bagus pas the best bahan kaku,negative,-0.4
18,"bagus banget ga nyangka, cuman sayang di aku t...",bintang 4,2 bulan lalu,bagus banget ga nyangka cuman sayang di aku te...,bagus banget ga nyangka cuman sayang di aku te...,bagus banget gak nyangka cuman sayang di saya ...,"[bagus, banget, gak, nyangka, cuman, sayang, d...","[bagus, banget, gak, nyangka, sayang, kesempit...",bagus banget gak nyangka sayang kesempitan dik...,negative,-1.8


In [78]:
preprocessed_df[preprocessed_df['label']=='neutral'].head()

,review,rating,date,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,label,polarity_score
0,"Bagus banget sesuai banget sama uk badan aku ,...",bintang 5,4 minggu lalu,Bagus banget sesuai banget sama uk badan aku b...,bagus banget sesuai banget sama uk badan aku b...,bagus banget sesuai banget sama uk badan saya ...,"[bagus, banget, sesuai, banget, sama, uk, bada...","[bagus, banget, sesuai, banget, uk, badan, bah...",bagus banget sesuai banget uk badan bahan paka...,neutral,-2.000000e-01
5,"Cantik sekali, 😍😍😍sesuai barang nya 🥰",bintang 5,1 minggu lalu,Cantik sekali sesuai barang nya,cantik sekali sesuai barang nya,cantik sekali sesuai barang nya,"[cantik, sekali, sesuai, barang, nya]","[cantik, sesuai, barang]",cantik sesuai barang,neutral,2.000000e-01
7,Kok bisa belehan ya belakang nya sy kira nggk ...,bintang 3,3 bulan lalu,Kok bisa belehan ya belakang nya sy kira nggk ...,kok bisa belehan ya belakang nya sy kira nggk ...,kok bisa belehan iya belakang nya saya kira ng...,"[kok, bisa, belehan, iya, belakang, nya, saya,...","[belehan, nggk, apa-apa]",belehan nggk apa-apa,neutral,0.000000e+00
9,"bagus,wrna cream,uk standar,pas",bintang 4,3 bulan lalu,baguswrna creamuk standarpas,baguswrna creamuk standarpas,baguswrna creamuk standarpas,"[baguswrna, creamuk, standarpas]","[baguswrna, creamuk, standarpas]",baguswrna creamuk standarpas,neutral,0.000000e+00
17,Sangat bagus sekali sesuai dengan gambar datan...,bintang 5,5 hari lalu,Sangat bagus sekali sesuai dengan gambar datan...,sangat bagus sekali sesuai dengan gambar datan...,sangat bagus sekali sesuai dengan gambar datan...,"[sangat, bagus, sekali, sesuai, dengan, gambar...","[bagus, sesuai, gambar, x]",bagus sesuai gambar x,neutral,-1.110223e-16


In [79]:
preprocessed_df[preprocessed_df['label']=='positive'].head()

,review,rating,date,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,label,polarity_score
1,bagus banget paketnya aku sangat suka aku kira...,bintang 5,2 bulan lalu,bagus banget paketnya aku sangat suka aku kira...,bagus banget paketnya aku sangat suka aku kira...,bagus banget paketnya saya sangat suka saya ki...,"[bagus, banget, paketnya, saya, sangat, suka, ...","[bagus, banget, paketnya, suka, rok, gak, kare...",bagus banget paketnya suka rok gak karetnya pa...,positive,1.2
2,Untuk bahannya nyaman enak dan pas untuk tingg...,bintang 5,2 bulan lalu,Untuk bahannya nyaman enak dan pas untuk tingg...,untuk bahannya nyaman enak dan pas untuk tingg...,untuk bahannya nyaman enak dan pas untuk tingg...,"[untuk, bahannya, nyaman, enak, dan, pas, untu...","[bahannya, nyaman, enak, pas, si, panjangan, n...",bahannya nyaman enak pas si panjangan ngk papa...,positive,3.2
3,"UK pass, cuma agak tipis bahannyaa sesuai seeh...",bintang 5,3 bulan lalu,UK pass cuma agak tipis bahannyaa sesuai seehh...,uk pass cuma agak tipis bahannyaa sesuai seehh...,uk pass cuma agak tipis bahannyaa sesuai seehh...,"[uk, pass, cuma, agak, tipis, bahannyaa, sesua...","[uk, pass, tipis, bahannyaa, sesuai, seehh, ha...",uk pass tipis bahannyaa sesuai seehh harga seg...,positive,1.4
6,"barang nya bagus, sesuai harga, warna nya juga...",bintang 5,2 bulan lalu,barang nya bagus sesuai harga warna nya juga b...,barang nya bagus sesuai harga warna nya juga b...,barang nya bagus sesuai harga warna nya juga b...,"[barang, nya, bagus, sesuai, harga, warna, nya...","[barang, bagus, sesuai, harga, warna, bagus, c...",barang bagus sesuai harga warna bagus cream,positive,0.4
8,Paketnya sudah sampai bagus dan pas banget,bintang 5,2 bulan lalu,Paketnya sudah sampai bagus dan pas banget,paketnya sudah sampai bagus dan pas banget,paketnya sudah sampai bagus dan pas banget,"[paketnya, sudah, sampai, bagus, dan, pas, ban...","[paketnya, bagus, pas, banget]",paketnya bagus pas banget,positive,0.4


In [69]:
# Add label to the initial df
labeled_df = preprocessed_df.drop(columns=['text_clean', 'text_casefoldingText', 'text_slangwords', 'text_tokenizingText', 'text_stopword', 'review'])

labeled_df.head()

,rating,date,text_akhir,label
0,bintang 5,4 minggu lalu,bagus banget sesuai banget uk badan bahan paka...,neutral
1,bintang 5,2 bulan lalu,bagus banget paketnya suka rok gak karetnya pa...,positive
2,bintang 5,2 bulan lalu,bahannya nyaman enak pas si panjangan ngk papa...,positive
3,bintang 5,3 bulan lalu,uk pass tipis bahannyaa sesuai seehh harga seg...,positive
4,bintang 4,1 bulan lalu,trnyata bahan melarbahan e kasarmakanya hrga m...,negative


### Checking the label distribution

In [70]:
print(labeled_df['label'].value_counts())

label
positive    5072
negative    3484
neutral     2162
Name: count, dtype: int64


In [72]:
labeled_df[labeled_df['label']=='negative'].head()

,rating,date,text_akhir,label
4,bintang 4,1 bulan lalu,trnyata bahan melarbahan e kasarmakanya hrga m...,negative
10,bintang 1,2 bulan lalu,pesen mengecewakan,negative
11,bintang 5,3 bulan lalu,bahannya bagus harganya terjangkau kegedean ka...,negative
14,bintang 4,3 bulan lalu,bagus pas the best bahan kaku,negative
18,bintang 4,2 bulan lalu,bagus banget gak nyangka sayang kesempitan dik...,negative


In [71]:
labeled_df[labeled_df['label']=='neutral'].head()

,rating,date,text_akhir,label
0,bintang 5,4 minggu lalu,bagus banget sesuai banget uk badan bahan paka...,neutral
5,bintang 5,1 minggu lalu,cantik sesuai barang,neutral
7,bintang 3,3 bulan lalu,belehan nggk apa-apa,neutral
9,bintang 4,3 bulan lalu,baguswrna creamuk standarpas,neutral
17,bintang 5,5 hari lalu,bagus sesuai gambar x,neutral


In [73]:
labeled_df[labeled_df['label']=='positive'].head()

,rating,date,text_akhir,label
1,bintang 5,2 bulan lalu,bagus banget paketnya suka rok gak karetnya pa...,positive
2,bintang 5,2 bulan lalu,bahannya nyaman enak pas si panjangan ngk papa...,positive
3,bintang 5,3 bulan lalu,uk pass tipis bahannyaa sesuai seehh harga seg...,positive
6,bintang 5,2 bulan lalu,barang bagus sesuai harga warna bagus cream,positive
8,bintang 5,2 bulan lalu,paketnya bagus pas banget,positive


### Balancing DF

In [ ]:
# Separate classes
df_0 = df[df['sentiment_label'] == 0]
df_1 = df[df['sentiment_label'] == 1]
df_2 = df[df['sentiment_label'] == 2]

# Downsample class 2
df_2_downsampled = resample(df_2,
                            replace=False,         # without replacement
                            n_samples=min(len(df_0), len(df_1)),  # match the minority class
                            random_state=42)

# Combine
df_balanced = pd.concat([df_0, df_1, df_2_downsampled])

# Shuffle the new balanced dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# View new distribution
print(df_balanced['sentiment_label'].value_counts())